# Loading several LoRA models

In [ ]:
! nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

In [ ]:
! pip install peft xformers bitsandbytes transformers accelerate -q

In [ ]:
! pip install git+https://github.com/huggingface/diffusers.git -q

In [ ]:
import torch
from diffusers import DiffusionPipeline
from diffusers import EulerAncestralDiscreteScheduler

import warnings
warnings.filterwarnings('ignore')

## Stable Diffusion 1.5

###  LoRA TempestMagic 
https://civitai.com/models/330933?modelVersionId=370779

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

pipe.load_lora_weights("civitai_sd15_loras", weight_name="TempestMagic-21.safetensors")
pipe.scheduler=EulerAncestralDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")

Usando un prompt normal podemos ya ver "estilo" parecido

In [ ]:
image = pipe("A book, very detailed, ultradefined, artstation, trending on artstation, 8k, HQ, sharp focus", guidance_scale=7.5, num_inference_steps=30).images[0]
image

Pero en Civitai podemos ver que hay ciertas palabras que lo "activan" (Trigger Words) más como: `tempestmagic, fantasy, ethereal transparent, motion blur`

In [ ]:
image = pipe("A book tempestmagic, fantasy, ethereal transparent, motion blur, very detailed, ultradefined, artstation, trending on artstation, 8k, HQ, sharp focus", guidance_scale=7.5, num_inference_steps=30).images[0]
image

In [ ]:
image = pipe("A portrait of a woman tempestmagic, very detailed, ultradefined, artstation, trending on artstation, 8k, HQ, sharp focus, ultra detailed, cinematic lighting, artistic", guidance_scale=9, num_inference_steps=30).images[0]
image

### LoRA Cyberpunk
https://civitai.com/models/128568?modelVersionId=140788

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

pipe.load_lora_weights("civitai_sd15_loras", weight_name="Cyberpunk_Anime-10.safetensors")
pipe.scheduler=EulerAncestralDiscreteScheduler.from_pretrained("runwayml/stable-diffusion-v1-5", subfolder="scheduler")

In [ ]:
image = pipe("A portrait of a space adventurer woman, very detailed, ultradefined, artstation, trending on artstation, 8k, HQ, sharp focus, ultra detailed, cinematic lighting, artistic", guidance_scale=9, num_inference_steps=30).images[0]
image

Añadiendo su Triger Word: `Cyberpunk_Anime`

In [ ]:
image = pipe("A portrait of a space adventurer woman, Cyberpunk_Anime, very detailed, ultradefined, artstation, trending on artstation, 8k, HQ, sharp focus, ultra detailed, cinematic lighting, artistic", guidance_scale=9, num_inference_steps=30).images[0]
image

## Load several LoRA models and merge it!
https://huggingface.co/docs/diffusers/using-diffusers/merge_loras

In [ ]:
pipe = DiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", torch_dtype=torch.float16).to("cuda")

In [ ]:
# Load LoRAs and set the adapter names
pipe.load_lora_weights("civitai_sd15_loras", weight_name="TempestMagic-21.safetensors", adapter_name="magic")
pipe.load_lora_weights("civitai_sd15_loras", weight_name="Cyberpunk_Anime-10.safetensors", adapter_name="cyber")

In [ ]:
# Set adapter weights
pipe.set_adapters(["magic", "cyber"], adapter_weights=[0.9, 0.3])

In [ ]:
image = pipe("A portrait of an space adventurer woman, very detailed, ultradefined, artstation, trending on artstation, 8k, HQ, sharp focus, ultra detailed, cinematic lighting, artistic", guidance_scale=9, num_inference_steps=30).images[0]
image

---

## SDXL

In [ ]:
from diffusers import DiffusionPipeline
import torch

from diffusers import DiffusionPipeline, AutoencoderKL

vae = AutoencoderKL.from_pretrained("madebyollin/sdxl-vae-fp16-fix", torch_dtype=torch.float16)
pipexl = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    vae=vae, torch_dtype=torch.float16, variant="fp16",
    use_safetensors=True
).to("cuda")

In [ ]:
pipexl.load_lora_weights("civitai_sdxl_loras", weight_name="Tempest_Magic_-_XL_Grimoire.safetensors")
pipexl.scheduler=EulerAncestralDiscreteScheduler.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", subfolder="scheduler")

In [ ]:
image = pipe("A book tempestmagic, ethereal transparent, motion blur", guidance_scale=7.5, num_inference_steps=30).images[0]
image

---